# environment setting

In [ ]:
import tensorflow as tf

print(tf.__version__)

In [ ]:
!pip install -U --pre tensorflow=="1.15.*"

In [ ]:
%%bash
cd models/research/slim
pip install .
cd models/research/object_detection/packages/tf1
pip install .

In [ ]:
%set_env PYTHONPATH=models/research:models/research/slim
import os
os.chdir('models/research/object_detection')

# create csv file

In [ ]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    for directory in ['PATH/xml/train', 'PATH/xml/test']:
        image_path = os.path.join(os.getcwd(), '{}'.format(directory))
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv('TRAINING_FOLDER_PATH/{}_labels.csv'.format(directory.split('/')[-1]), index=None) # save in TRAINING_FOLDER_PATH
        print('Successfully converted xml to csv.')

main()

# create record file

In [ ]:
!python PATH/generate_tfrecord.py --csv_input=TRAINING_FOLDER_PATH/train_labels.csv --output_path=TRAINING_FOLDER_PATH/train.record --image_dir=PATH/image/train
!python PATH/generate_tfrecord.py --csv_input=TRAINING_FOLDER_PATH/test_labels.csv --output_path=TRAINING_FOLDER_PATH/test.record --image_dir=PATH/image/test

# train

In [ ]:
!python legacy/train.py --logtostderr --train_dir=training --pipeline_config_path=training/rfcn_resnet101_coco.config